In [1]:
import pandas as pd
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
import pyarrow
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
file_name = "allo_cine"
version="2.2.1"

ml_client = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id="72eb7803-e874-44cb-b6d9-33f2fa3eb88c",
    resource_group_name="vpoutotrg",
    workspace_name="mlstudio-groupe4"
)

data_asset = ml_client.data.get(file_name, version=version)
print(data_asset.path)


df = pd.read_parquet(data_asset.path)

df.head()


azureml://subscriptions/72eb7803-e874-44cb-b6d9-33f2fa3eb88c/resourcegroups/vpoutotrg/workspaces/mlstudio-groupe4/datastores/workspaceblobstore/paths/LocalUpload/19ec33ce8858cdb90c86accbe1c1a507/allocine_spider_clean.parquet


Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


,actors,critics_score,date,directors,editor,genre,langage,length,nationality,viewers_score,us_first_week_boxoffice,french_first_week_boxoffice,french_visa,title,vo_title,url
0,"[Arieh Worthalter, Arthur Harari, Stéphan Guér...",4.4,2023-09-27,[Cédric Kahn],Ad Vitam,"[Policier, Drame, Historique, Judiciaire]",[Français],116.0,[France],3.9,NaN,110011.0,157303,Le Procès Goldman,None,/article/fichearticle_gen_carticle=1000093547....
1,"[Monica Bellucci, Vincent Cassel, Albert Dupon...",3.5,2020-08-26,[Gaspar Noé],Carlotta Films,"[Drame, Thriller]","[Anglais, Français, Italien, Espagnol]",90.0,[France],NaN,NaN,2905.0,153336,Irréversible - Inversion Intégrale,None,/article/fichearticle_gen_carticle=18692477.html
2,[Thom Hoffman],2.8,2024-08-07,"[Richard Claus, Karsten Kiilerich]",Le Pacte,"[Aventure, Animation, Comédie, Famille]",[Néerlandais],84.0,"[Danemark, France, Allemagne, Pays-Bas]",3.0,NaN,39119.0,160622,Petit Panda en Afrique,Panda Bear in Africa,/article/fichearticle_gen_carticle=1000096064....
3,"[Lou de Laâge, Raphaël Personnaz, Isabelle Car...",3.3,2022-12-21,[Olivier Treiner],SND,[Drame],[Français],120.0,[France],3.8,NaN,64039.0,152607,Le Tourbillon de la vie,None,/article/fichearticle_gen_carticle=1000006465....
4,"[Michael B. Jordan, Jamie Foxx, Brie Larson, R...",3.0,2020-01-29,[Destin Daniel Cretton],Warner Bros. France,"[Biopic, Drame]",[Anglais],137.0,[U.S.A.],4.1,9713228.0,113153.0,152118,La Voie de la justice,Just Mercy,/video/player_gen_cmedia=19586793&cfilm=239735...


In [3]:
table = pa.Table.from_pandas(df)
pq.write_table(table, 'allocine_spider_clean.parquet')

In [6]:
df['viewers_score']= pd.to_numeric(df['viewers_score'], errors='coerce')
df['critics_score']= pd.to_numeric(df['critics_score'], errors='coerce')
# 
#df['french_boxoffice']= pd.to_numeric(df['french_boxoffice'], errors='coerce')

In [7]:
df['length']= pd.to_numeric(df['length'], errors='coerce')
df['length'] = df['length'].apply(lambda x : np.nan if x == 0 else x )

In [8]:

df['date']= pd.to_datetime(df['date'], errors='coerce')

In [9]:
df['genre'] = df['genre'].str.split('|')
df['actors'] = df['actors'].str.split('|')
df['nationality'] = df['nationality'].str.split('|')
df['langage'] = df['langage'].str.split('|') 	

In [10]:
df = df[df['french_first_week_boxoffice'].notna()]
df = df[df['date'].notna()]


In [12]:
feature_of_interest = [
    'actors',
    'critics_score',
    'date',
    'directors',
    'editor',
    'genre',
    'langage',
    'length',
    'nationality',
    'viewers_score'
]

info_film = ['french_visa', 'title', 'vo_title', 'url']


target = 'french_first_week_boxoffice'
data = feature_of_interest
data.append(target)

data.extend(info_film)
df_to_train = df[data]

In [13]:
df_to_train

,actors,critics_score,date,directors,editor,genre,langage,length,nationality,viewers_score,french_first_week_boxoffice,french_visa,title,vo_title,url
0,NaN,4.4,2023-09-27,[Cédric Kahn],Ad Vitam,NaN,NaN,116.0,NaN,3.9,110011.0,157303,Le Procès Goldman,None,/article/fichearticle_gen_carticle=1000093547....
1,NaN,3.5,2020-08-26,[Gaspar Noé],Carlotta Films,NaN,NaN,90.0,NaN,NaN,2905.0,153336,Irréversible - Inversion Intégrale,None,/article/fichearticle_gen_carticle=18692477.html
2,NaN,2.8,2024-08-07,"[Richard Claus, Karsten Kiilerich]",Le Pacte,NaN,NaN,84.0,NaN,3.0,39119.0,160622,Petit Panda en Afrique,Panda Bear in Africa,/article/fichearticle_gen_carticle=1000096064....
3,NaN,3.3,2022-12-21,[Olivier Treiner],SND,NaN,NaN,120.0,NaN,3.8,64039.0,152607,Le Tourbillon de la vie,None,/article/fichearticle_gen_carticle=1000006465....
4,NaN,3.0,2020-01-29,[Destin Daniel Cretton],Warner Bros. France,NaN,NaN,137.0,NaN,4.1,113153.0,152118,La Voie de la justice,Just Mercy,/video/player_gen_cmedia=19586793&cfilm=239735...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9153,NaN,3.2,2022-08-03,[David Leitch],Sony Pictures Releasing France,NaN,NaN,127.0,NaN,3.9,491067.0,157053,Bullet Train,None,/video/player_gen_cmedia=19596933&cfilm=284846...
9154,NaN,3.8,2024-10-30,[Clint Eastwood],Warner Bros. France,NaN,NaN,114.0,NaN,3.9,453813.0,-,Juré n°2,Juror #2,/video/player_gen_cmedia=20609934&cfilm=314868...
9155,NaN,3.5,2022-05-11,[François Uzan],UGC Distribution,NaN,NaN,96.0,NaN,2.8,37740.0,153600,On sourit pour la photo,None,/article/fichearticle_gen_carticle=18709697.html
9156,NaN,3.6,2024-05-01,"[Juan Sebastián Vásquez, Alejandro Rojas]",Condor Distribution,NaN,NaN,77.0,NaN,3.8,44476.0,161784,Border Line,Upon Entry (La llegada),/video/player_gen_cmedia=20601439&cfilm=309465...


In [14]:
table = pa.Table.from_pandas(df_to_train)
pq.write_table(table, 'allocine_spider_clean.parquet')

In [4]:
df2 = pd.read_csv("ecran_total_spider.csv")
df2.head(5)


,date,genre,title,url_picture
0,16/04/2025,"Biopic, Drame",OXANA,https://ecran-total.fr/wp-content/uploads/2023...
1,16/04/2025,Comédie dramatique,LA RÉPARATION,https://ecran-total.fr/wp-content/uploads/2024...
2,16/04/2025,"Drame, Western",HARVEST,https://ecran-total.fr/wp-content/uploads/2024...
3,16/04/2025,Comédie,LE MÉLANGE DES GENRES,https://ecran-total.fr/wp-content/uploads/2024...
4,16/04/2025,"Drame, Horreur, Thriller",SINNERS,https://ecran-total.fr/wp-content/uploads/2024...


In [5]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         276 non-null    object
 1   genre        276 non-null    object
 2   title        276 non-null    object
 3   url_picture  276 non-null    object
dtypes: object(4)
memory usage: 8.8+ KB


In [10]:
df2['date']=pd.to_datetime(df2['date'], errors = 'coerce')
df2['year']=df2['date'].dt.year

/tmp/ipykernel_75859/711646458.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df2['date']=pd.to_datetime(df2['date'], errors = 'coerce')


In [12]:
df2.head(20)

,date,genre,title,url_picture,year
0,2025-04-16,"Biopic, Drame",OXANA,https://ecran-total.fr/wp-content/uploads/2023...,2025
1,2025-04-16,Comédie dramatique,LA RÉPARATION,https://ecran-total.fr/wp-content/uploads/2024...,2025
2,2025-04-16,"Drame, Western",HARVEST,https://ecran-total.fr/wp-content/uploads/2024...,2025
3,2025-04-16,Comédie,LE MÉLANGE DES GENRES,https://ecran-total.fr/wp-content/uploads/2024...,2025
4,2025-04-16,"Drame, Horreur, Thriller",SINNERS,https://ecran-total.fr/wp-content/uploads/2024...,2025
5,2025-04-16,Non classé,KYUKA – AVANT LA FIN DE L’ÉTÉ,https://ecran-total.fr/wp-content/uploads/2024...,2025
6,2025-04-16,Action,RAPIDE,https://ecran-total.fr/wp-content/uploads/2024...,2025
7,2025-04-16,Comédie,AIMONS-NOUS VIVANTS,https://ecran-total.fr/wp-content/uploads/2024...,2025
8,2025-04-16,Drame,TOXIC,https://ecran-total.fr/wp-content/uploads/2024...,2025
9,2025-04-16,Drame,DIMANCHES,https://ecran-total.fr/wp-content/uploads/2024...,2025
